<h3>Ricerca iperparametri tramite GridSearchCV</h3>

<p>Iniziamo con l'addestrare alcuni semplici modelli e ricercare la configurazione migliore di iperparametri. Successivamente confronteremo i risultati ottenuti con quelli risultanti da pipelines più complesse cercando di spiegare e motivare i possibili miglioramenti osservati.

In [40]:
import os
import time
import re
import pickle
import numpy as np
import pandas as pd

#Libreria di ML per Python, oltre ad offrire innumerevoli implementazioni di modelli predittivi offre anche
#una vasta gamma di funzionalità per il data cleaning, feature selection, feature extraction, etc...
#Per ora impiegheremo la classe CountVectorizer per creare la 'bag of words'.
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.pipeline import Pipeline

from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeRegressor, export_text
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDRegressor, LinearRegression


from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

#Per poter utilizzare la lista di stopwords di NLTK è necessario scaricarla!
#import nltk
#nltk.download()

#Libreria per generare grafici
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, LabelSet, FactorRange

#Funzione invocata per redirigere l'output della libreria
#Bokeh in modo da visualizzare i grafici nel notebook, se omesso i grafici sono generati sotto forma
#di file HTML
output_notebook()

SEED = 99
np.random.seed(SEED)

DATA_FOLDER = os.path.join(os.getcwd(), "../data")
RESULTS_FOLDER = os.path.join(DATA_FOLDER, "results")
CV_RESULTS_FOLDER = os.path.join(RESULTS_FOLDER, "csv")

Loading BokehJS ...

<h3>Lettura Dati</h3>

In [5]:
train_fp = os.path.join(os.getcwd(), "../data/csv/train.csv")
train = pd.read_csv(
    train_fp,
    index_col=['review_id']
)

In [6]:
test_fp = os.path.join(os.getcwd(), "../data/csv/train.csv")
test = pd.read_csv(
    test_fp,
    index_col=['review_id']
)

<h3>Separazione feature-labels</h3>

In [7]:
X_train, Y_train = train['text'], train['stars']
X_test, Y_test = test['text'], test['stars']

<h3>Funzioni ausiliarie</h3>

In [35]:
def clean_results_df(df, to_remove, to_format, format_, new_index_cols):
    """
        Clean results dataframe contaning GridSearch error metrics.

        Params :
            df -> pandas.DataFrame : DataFrame to clean.
            to_remove -> python.dict : dictionary of elements to remove.
            new_index_cols -> python.list : list of columns to use as new index
    """
    df = df.copy()
    for k,v in to_remove.items():
        if v is True:
            df = df.drop(k, axis=1)
        elif isinstance(v, list):
            df = df.drop(k, 1).assign(**pd.DataFrame(df[k].values.tolist()))
            for col in v:
                df = df.drop(col, 1)
    labels = [df[col] for col in new_index_cols]
    labels = list(zip(*labels))
    new_index = pd.MultiIndex.from_tuples(labels, names=new_index_cols)
    df = df.set_index(new_index)
    df = df.drop(new_index_cols, 1)
    for col in to_format:
        df[col] = df[col].apply(lambda x : format(x, format_))
    return df

def dump_results(df, fp):
    df.to_csv(fp)

<h3>Definizione della Pipeline</h3>

In [6]:
simple_pipeline = Pipeline([
    ('cv', CountVectorizer()),
    ('model', 'passthrough') 
])

<h3>Griglia iperparametri</h3>

In [7]:
simple_param_grid = [
    {
        #SGD
        'model':[SGDRegressor()],
        'model__penalty':['l2','l1'],
        'model__alpha':[0.00001, 0.0001, 0.001],
        'model__learning_rate':['constant', 'optimal'],
        'model__max_iter':[2500]
    },
    {
        #Linear
        'model':[LinearRegression()],
        'model__normalize':[True, False]
    },
    {
        #DTR
        'model':[DecisionTreeRegressor()]
    }
]

<h3>Training</h3>

In [14]:
simple_gs = GridSearchCV(
    simple_pipeline,
    param_grid=simple_param_grid,
    scoring=['neg_mean_absolute_error', 'neg_root_mean_squared_error'],
    n_jobs=-1,
    cv=5,
    refit=False
)

In [15]:
simple_gs.fit(X_train, Y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('cv', CountVectorizer()),
                                       ('model', 'passthrough')]),
             n_jobs=-1,
             param_grid=[{'model': [SGDRegressor()],
                          'model__alpha': [1e-05, 0.0001, 0.001],
                          'model__learning_rate': ['constant', 'optimal'],
                          'model__max_iter': [2500],
                          'model__penalty': ['l2', 'l1']},
                         {'model': [LinearRegression()],
                          'model__normalize': [True, False]},
                         {'model': [DecisionTreeRegressor()]}],
             refit=False,
             scoring=['neg_mean_absolute_error', 'neg_root_mean_squared_error'])

<h3>Dump risultati GridSearchCV</h3>

In [40]:
results_df = clean_results_df(
    pd.DataFrame(simple_gs.cv_results_),
    to_remove={},
    to_format=['mean_test_neg_mean_absolute_error', 'mean_test_neg_root_mean_squared_error'],
    format_=".8f",
    new_index_cols=['rank_test_neg_root_mean_squared_error', 'rank_test_neg_mean_absolute_error']
)

In [42]:
results_fp = os.path.join(CV_RESULTS_FOLDER, "simple_gs_results.csv")
dump_results(results_df, results_fp)